# Automatic Certificate Generator with Multiple Templates

This notebook provides a comprehensive solution for automatically generating certificates with:
- **Multiple certificate templates** for different types of certificates
- **Manual name entry** or **Excel file input** support
- **Customizable attributes** (name, course, date, grade, etc.)
- **Bulk generation** capabilities
- **Google Colab compatibility**

## Features:
- ✅ Multiple professional certificate templates
- ✅ Excel/CSV file input support
- ✅ Manual name entry option
- ✅ Customizable fonts and positions
- ✅ Batch processing
- ✅ Output in PNG and PDF formats
- ✅ Google Drive integration for Colab

**Author:** AI Engineering Student  
**Compatible with:** Google Colab, Jupyter Notebook  
**Libraries Used:** PIL (Pillow), pandas, reportlab

## 📦 Installation and Setup

First, let's install all required libraries and set up the environment.

In [ ]:
# Install required libraries
!pip install pillow pandas reportlab openpyxl xlrd

# Import necessary libraries
import os
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter, A4
from reportlab.lib.utils import ImageReader
import zipfile
from datetime import datetime
import io
from google.colab import files, drive
import urllib.request
import json

print("✅ All libraries installed successfully!")

## 🔗 Google Drive Integration

Mount Google Drive to save/load templates and output files.

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Create necessary directories
base_path = '/content/certificate_generator'
templates_path = f'{base_path}/templates'
fonts_path = f'{base_path}/fonts'
output_path = f'{base_path}/generated_certificates'
data_path = f'{base_path}/data'

# Create directories
for path in [base_path, templates_path, fonts_path, output_path, data_path]:
    os.makedirs(path, exist_ok=True)

print("📁 Directory structure created:")
print(f"📂 Base: {base_path}")
print(f"📂 Templates: {templates_path}")
print(f"📂 Fonts: {fonts_path}")
print(f"📂 Output: {output_path}")
print(f"📂 Data: {data_path}")

## 🎨 Certificate Template Creator

This section creates multiple certificate templates programmatically.

In [ ]:
def create_template(template_name, width=1200, height=800, bg_color='white'):
    """
    Create a certificate template with basic design elements
    """
    # Create a new image
    img = Image.new('RGB', (width, height), color=bg_color)
    draw = ImageDraw.Draw(img)
    
    # Draw border
    border_width = 20
    draw.rectangle([
        (border_width, border_width), 
        (width - border_width, height - border_width)
    ], outline='#2E4A6B', width=5)
    
    # Inner decorative border
    draw.rectangle([
        (border_width + 10, border_width + 10), 
        (width - border_width - 10, height - border_width - 10)
    ], outline='#D4AF37', width=2)
    
    if template_name == 'completion':
        # Certificate of Completion design
        draw.text((width//2, 120), "CERTIFICATE", font=None, fill='#2E4A6B', anchor='mm')
        draw.text((width//2, 170), "OF COMPLETION", font=None, fill='#2E4A6B', anchor='mm')
        draw.text((width//2, 250), "This is to certify that", font=None, fill='black', anchor='mm')
        draw.text((width//2, 350), "[PARTICIPANT_NAME]", font=None, fill='#D4AF37', anchor='mm')
        draw.text((width//2, 420), "has successfully completed the course", font=None, fill='black', anchor='mm')
        draw.text((width//2, 470), "[COURSE_NAME]", font=None, fill='#2E4A6B', anchor='mm')
        draw.text((width//2, 550), "on [DATE]", font=None, fill='black', anchor='mm')
    
    elif template_name == 'achievement':
        # Certificate of Achievement design
        draw.text((width//2, 120), "CERTIFICATE", font=None, fill='#8B0000', anchor='mm')
        draw.text((width//2, 170), "OF ACHIEVEMENT", font=None, fill='#8B0000', anchor='mm')
        draw.text((width//2, 250), "Presented to", font=None, fill='black', anchor='mm')
        draw.text((width//2, 350), "[PARTICIPANT_NAME]", font=None, fill='#FFD700', anchor='mm')
        draw.text((width//2, 420), "for outstanding performance in", font=None, fill='black', anchor='mm')
        draw.text((width//2, 470), "[COURSE_NAME]", font=None, fill='#8B0000', anchor='mm')
        draw.text((width//2, 520), "Grade: [GRADE]", font=None, fill='black', anchor='mm')
        draw.text((width//2, 580), "Date: [DATE]", font=None, fill='black', anchor='mm')
    
    elif template_name == 'participation':
        # Certificate of Participation design
        draw.text((width//2, 120), "CERTIFICATE", font=None, fill='#006400', anchor='mm')
        draw.text((width//2, 170), "OF PARTICIPATION", font=None, fill='#006400', anchor='mm')
        draw.text((width//2, 250), "This certifies that", font=None, fill='black', anchor='mm')
        draw.text((width//2, 350), "[PARTICIPANT_NAME]", font=None, fill='#32CD32', anchor='mm')
        draw.text((width//2, 420), "participated in", font=None, fill='black', anchor='mm')
        draw.text((width//2, 470), "[EVENT_NAME]", font=None, fill='#006400', anchor='mm')
        draw.text((width//2, 520), "held on [DATE]", font=None, fill='black', anchor='mm')
    
    # Save template
    template_path = f'{templates_path}/{template_name}_template.png'
    img.save(template_path)
    return template_path

# Create templates
templates = {}
for template_type in ['completion', 'achievement', 'participation']:
    path = create_template(template_type)
    templates[template_type] = path
    print(f"✅ Created {template_type} template: {path}")

print("\n🎨 All certificate templates created successfully!")

## 🔤 Font Management

Download and set up fonts for certificate generation.

In [ ]:
def download_fonts():
    """
    Download Google Fonts for certificate generation
    """
    font_urls = {
        'Roboto-Bold.ttf': 'https://github.com/google/fonts/raw/main/apache/roboto/Roboto-Bold.ttf',
        'Roboto-Regular.ttf': 'https://github.com/google/fonts/raw/main/apache/roboto/Roboto-Regular.ttf',
        'OpenSans-Bold.ttf': 'https://github.com/google/fonts/raw/main/apache/opensans/OpenSans-Bold.ttf',
        'OpenSans-Regular.ttf': 'https://github.com/google/fonts/raw/main/apache/opensans/OpenSans-Regular.ttf'
    }
    
    downloaded_fonts = {}
    
    for font_name, url in font_urls.items():
        font_path = f'{fonts_path}/{font_name}'
        if not os.path.exists(font_path):
            try:
                urllib.request.urlretrieve(url, font_path)
                print(f"✅ Downloaded: {font_name}")
            except:
                print(f"❌ Failed to download: {font_name}")
                continue
        else:
            print(f"✅ Font already exists: {font_name}")
        
        downloaded_fonts[font_name.split('.')[0]] = font_path
    
    return downloaded_fonts

# Download fonts
available_fonts = download_fonts()
print(f"\n🔤 Available fonts: {list(available_fonts.keys())}")

## 📊 Data Input Methods

Support for both manual entry and Excel/CSV file input.

In [ ]:
def create_sample_data():
    """
    Create sample Excel files for demonstration
    """
    # Sample data for different certificate types
    sample_data = {
        'completion': {
            'Participant_Name': ['John Doe', 'Jane Smith', 'Mike Johnson', 'Sarah Williams', 'David Brown'],
            'Course_Name': ['Python Programming', 'Data Science', 'Machine Learning', 'Web Development', 'AI Fundamentals'],
            'Date': ['2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '2024-10-05'],
            'Instructor': ['Dr. Smith', 'Prof. Johnson', 'Dr. Lee', 'Prof. Davis', 'Dr. Wilson']
        },
        'achievement': {
            'Participant_Name': ['Alice Cooper', 'Bob Taylor', 'Carol White', 'Dan Miller', 'Eve Garcia'],
            'Course_Name': ['Advanced Python', 'Deep Learning', 'Data Analytics', 'Cloud Computing', 'Cybersecurity'],
            'Grade': ['A+', 'A', 'A-', 'B+', 'A'],
            'Date': ['2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '2024-10-05'],
            'Score': [95, 92, 88, 85, 90]
        },
        'participation': {
            'Participant_Name': ['Tom Wilson', 'Lisa Anderson', 'Mark Davis', 'Amy Chen', 'Paul Martinez'],
            'Event_Name': ['AI Workshop', 'Tech Conference', 'Coding Bootcamp', 'Data Summit', 'ML Symposium'],
            'Date': ['2024-10-01', '2024-10-02', '2024-10-03', '2024-10-04', '2024-10-05'],
            'Duration': ['8 hours', '2 days', '1 week', '3 days', '4 hours']
        }
    }
    
    # Create Excel files
    for cert_type, data in sample_data.items():
        df = pd.DataFrame(data)
        excel_path = f'{data_path}/sample_{cert_type}_data.xlsx'
        df.to_excel(excel_path, index=False)
        print(f"✅ Created sample data: {excel_path}")
        print(f"   Columns: {list(df.columns)}")
        print(f"   Records: {len(df)}")
        print()

# Create sample data
create_sample_data()
print("📊 Sample data files created successfully!")

## 🎯 Certificate Generator Class

Main class for generating certificates with multiple templates and customization options.

In [ ]:
class CertificateGenerator:
    def __init__(self, templates_path, fonts_path, output_path):
        self.templates_path = templates_path
        self.fonts_path = fonts_path
        self.output_path = output_path
        self.available_fonts = available_fonts
        
        # Certificate configurations
        self.configs = {
            'completion': {
                'fields': {
                    'PARTICIPANT_NAME': {'position': (600, 350), 'size': 48, 'color': '#D4AF37', 'font': 'Roboto-Bold'},
                    'COURSE_NAME': {'position': (600, 470), 'size': 36, 'color': '#2E4A6B', 'font': 'Roboto-Bold'},
                    'DATE': {'position': (600, 550), 'size': 24, 'color': 'black', 'font': 'Roboto-Regular'}
                }
            },
            'achievement': {
                'fields': {
                    'PARTICIPANT_NAME': {'position': (600, 350), 'size': 48, 'color': '#FFD700', 'font': 'Roboto-Bold'},
                    'COURSE_NAME': {'position': (600, 470), 'size': 36, 'color': '#8B0000', 'font': 'Roboto-Bold'},
                    'GRADE': {'position': (600, 520), 'size': 28, 'color': 'black', 'font': 'Roboto-Regular'},
                    'DATE': {'position': (600, 580), 'size': 24, 'color': 'black', 'font': 'Roboto-Regular'}
                }
            },
            'participation': {
                'fields': {
                    'PARTICIPANT_NAME': {'position': (600, 350), 'size': 48, 'color': '#32CD32', 'font': 'Roboto-Bold'},
                    'EVENT_NAME': {'position': (600, 470), 'size': 36, 'color': '#006400', 'font': 'Roboto-Bold'},
                    'DATE': {'position': (600, 520), 'size': 24, 'color': 'black', 'font': 'Roboto-Regular'}
                }
            }
        }
    
    def get_font(self, font_name, size):
        """Get font object"""
        try:
            if font_name in self.available_fonts:
                return ImageFont.truetype(self.available_fonts[font_name], size)
            else:
                return ImageFont.load_default()
        except:
            return ImageFont.load_default()
    
    def generate_single_certificate(self, template_type, data_row, output_filename=None):
        """Generate a single certificate"""
        # Load template
        template_path = f'{self.templates_path}/{template_type}_template.png'
        if not os.path.exists(template_path):
            raise FileNotFoundError(f"Template not found: {template_path}")
        
        # Open template image
        img = Image.open(template_path).copy()
        draw = ImageDraw.Draw(img)
        
        # Get configuration for this template type
        config = self.configs.get(template_type, {})
        fields = config.get('fields', {})
        
        # Fill in the fields
        for field_name, field_config in fields.items():
            # Map field names to data columns
            data_key = self._map_field_to_data(field_name, data_row)
            
            if data_key and data_key in data_row:
                text = str(data_row[data_key])
                position = field_config['position']
                size = field_config['size']
                color = field_config['color']
                font_name = field_config['font']
                
                font = self.get_font(font_name, size)
                
                # Center the text
                bbox = draw.textbbox((0, 0), text, font=font)
                text_width = bbox[2] - bbox[0]
                centered_x = position[0] - text_width // 2
                
                draw.text((centered_x, position[1]), text, font=font, fill=color)
        
        # Generate output filename if not provided
        if not output_filename:
            participant_name = data_row.get('Participant_Name', 'Unknown')
            safe_name = ''.join(c for c in participant_name if c.isalnum() or c in (' ', '-', '_')).rstrip()
            output_filename = f"{template_type}_{safe_name.replace(' ', '_')}.png"
        
        # Save certificate
        output_path = f'{self.output_path}/{output_filename}'
        img.save(output_path)
        
        return output_path
    
    def _map_field_to_data(self, field_name, data_row):
        """Map template field names to data column names"""
        mapping = {
            'PARTICIPANT_NAME': 'Participant_Name',
            'COURSE_NAME': 'Course_Name',
            'EVENT_NAME': 'Event_Name',
            'DATE': 'Date',
            'GRADE': 'Grade'
        }
        return mapping.get(field_name)
    
    def generate_from_excel(self, excel_path, template_type, sheet_name=None):
        """Generate certificates from Excel file"""
        # Read Excel file
        try:
            if sheet_name:
                df = pd.read_excel(excel_path, sheet_name=sheet_name)
            else:
                df = pd.read_excel(excel_path)
        except Exception as e:
            raise Exception(f"Error reading Excel file: {str(e)}")
        
        generated_files = []
        
        # Generate certificate for each row
        for index, row in df.iterrows():
            try:
                output_path = self.generate_single_certificate(template_type, row)
                generated_files.append(output_path)
                print(f"✅ Generated certificate {index + 1}/{len(df)}: {os.path.basename(output_path)}")
            except Exception as e:
                print(f"❌ Error generating certificate for row {index + 1}: {str(e)}")
        
        return generated_files
    
    def generate_manual(self, template_type, participant_data):
        """Generate certificate from manually entered data"""
        return self.generate_single_certificate(template_type, participant_data)
    
    def convert_to_pdf(self, image_paths, pdf_output_path=None):
        """Convert generated certificate images to PDF"""
        if not pdf_output_path:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            pdf_output_path = f'{self.output_path}/certificates_{timestamp}.pdf'
        
        c = canvas.Canvas(pdf_output_path, pagesize=A4)
        
        for img_path in image_paths:
            if os.path.exists(img_path):
                # Add image to PDF
                img = ImageReader(img_path)
                c.drawImage(img, 0, 0, width=A4[0], height=A4[1])
                c.showPage()
        
        c.save()
        return pdf_output_path
    
    def create_zip_archive(self, file_paths, zip_name=None):
        """Create ZIP archive of generated certificates"""
        if not zip_name:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_name = f'certificates_{timestamp}.zip'
        
        zip_path = f'{self.output_path}/{zip_name}'
        
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for file_path in file_paths:
                if os.path.exists(file_path):
                    zipf.write(file_path, os.path.basename(file_path))
        
        return zip_path

# Initialize the certificate generator
cert_generator = CertificateGenerator(templates_path, fonts_path, output_path)
print("🎯 Certificate Generator initialized successfully!")

## 📝 Manual Certificate Generation

Generate certificates by manually entering participant data.

In [ ]:
# Manual certificate generation example
def generate_manual_certificate():
    print("📝 Manual Certificate Generation")
    print("Available templates: completion, achievement, participation")
    
    # Example data - you can modify this
    manual_data = {
        'completion': {
            'Participant_Name': 'John Doe',
            'Course_Name': 'Advanced Python Programming',
            'Date': '2024-10-03'
        },
        'achievement': {
            'Participant_Name': 'Jane Smith',
            'Course_Name': 'Machine Learning Fundamentals',
            'Grade': 'A+',
            'Date': '2024-10-03'
        },
        'participation': {
            'Participant_Name': 'Mike Johnson',
            'Event_Name': 'AI Tech Conference 2024',
            'Date': '2024-10-03'
        }
    }
    
    generated_files = []
    
    # Generate certificates for each type
    for template_type, data in manual_data.items():
        try:
            output_path = cert_generator.generate_manual(template_type, data)
            generated_files.append(output_path)
            print(f"✅ Generated {template_type} certificate: {os.path.basename(output_path)}")
        except Exception as e:
            print(f"❌ Error generating {template_type} certificate: {str(e)}")
    
    return generated_files

# Generate manual certificates
manual_certificates = generate_manual_certificate()
print(f"\n📊 Generated {len(manual_certificates)} manual certificates")

## 📊 Excel File Certificate Generation

Generate certificates in bulk from Excel/CSV files.

In [ ]:
# Bulk certificate generation from Excel files
def generate_from_excel_files():
    print("📊 Bulk Certificate Generation from Excel Files")
    
    # List available data files
    data_files = [f for f in os.listdir(data_path) if f.endswith(('.xlsx', '.csv'))]
    print(f"Available data files: {data_files}")
    
    all_generated_files = []
    
    # Generate certificates for each sample data file
    for file_name in data_files:
        if 'sample_' in file_name and file_name.endswith('.xlsx'):
            # Extract template type from filename
            template_type = file_name.replace('sample_', '').replace('_data.xlsx', '')
            
            if template_type in ['completion', 'achievement', 'participation']:
                print(f"\n🔄 Processing {file_name} for {template_type} certificates...")
                
                try:
                    excel_path = f'{data_path}/{file_name}'
                    generated_files = cert_generator.generate_from_excel(excel_path, template_type)
                    all_generated_files.extend(generated_files)
                    print(f"✅ Generated {len(generated_files)} certificates from {file_name}")
                except Exception as e:
                    print(f"❌ Error processing {file_name}: {str(e)}")
    
    return all_generated_files

# Generate certificates from Excel files
excel_certificates = generate_from_excel_files()
print(f"\n📊 Total certificates generated from Excel files: {len(excel_certificates)}")

## 📤 File Upload Interface

Upload your own Excel/CSV files for certificate generation.

In [ ]:
# File upload interface for custom data
def upload_and_process_custom_file():
    print("📤 Upload Your Own Data File")
    print("Supported formats: .xlsx, .csv")
    print("Required columns depend on certificate type:")
    print("- Completion: Participant_Name, Course_Name, Date")
    print("- Achievement: Participant_Name, Course_Name, Grade, Date")
    print("- Participation: Participant_Name, Event_Name, Date")
    
    try:
        # Upload file
        uploaded = files.upload()
        
        for filename, content in uploaded.items():
            # Save uploaded file
            file_path = f'{data_path}/{filename}'
            with open(file_path, 'wb') as f:
                f.write(content)
            
            print(f"✅ Uploaded file saved: {filename}")
            
            # Preview the data
            try:
                if filename.endswith('.xlsx'):
                    df = pd.read_excel(file_path)
                elif filename.endswith('.csv'):
                    df = pd.read_csv(file_path)
                else:
                    print("❌ Unsupported file format")
                    continue
                
                print(f"\n📊 Data Preview ({len(df)} rows):")
                print(f"Columns: {list(df.columns)}")
                print(df.head())
                
                # Ask user to specify template type
                print("\n🎯 Please specify the certificate template type in the next cell:")
                print("Available types: 'completion', 'achievement', 'participation'")
                
                return file_path, df
                
            except Exception as e:
                print(f"❌ Error reading uploaded file: {str(e)}")
    
    except Exception as e:
        print(f"❌ Error uploading file: {str(e)}")
        return None, None

# Uncomment the line below to upload your own file
# uploaded_file_path, uploaded_df = upload_and_process_custom_file()

print("📤 File upload interface ready. Uncomment the line above to upload your file.")

## 🔄 Process Uploaded File

Generate certificates from uploaded file (run this after uploading).

In [ ]:
# Process uploaded file (modify template_type as needed)
def process_uploaded_file(file_path, template_type):
    """
    Process uploaded file and generate certificates
    
    Args:
        file_path: Path to uploaded file
        template_type: 'completion', 'achievement', or 'participation'
    """
    if not file_path or not os.path.exists(file_path):
        print("❌ No file uploaded or file not found")
        return []
    
    print(f"🔄 Processing uploaded file with template type: {template_type}")
    
    try:
        generated_files = cert_generator.generate_from_excel(file_path, template_type)
        print(f"✅ Generated {len(generated_files)} certificates from uploaded file")
        return generated_files
    except Exception as e:
        print(f"❌ Error processing uploaded file: {str(e)}")
        return []

# Example usage (uncomment and modify after uploading a file):
# uploaded_certificates = process_uploaded_file(uploaded_file_path, 'completion')

print("🔄 Upload processor ready. Upload a file first, then run this cell with appropriate template type.")

## 📄 PDF Conversion and Archive Creation

Convert certificates to PDF and create downloadable archives.

In [ ]:
# Create PDF versions and archives
def create_pdf_and_archive():
    print("📄 Creating PDF versions and archives...")
    
    # Get all generated PNG certificates
    png_files = [f for f in os.listdir(output_path) if f.endswith('.png')]
    png_paths = [f'{output_path}/{f}' for f in png_files]
    
    if not png_files:
        print("❌ No PNG certificates found. Generate some certificates first.")
        return
    
    print(f"📊 Found {len(png_files)} PNG certificates")
    
    # Create PDF version
    try:
        pdf_path = cert_generator.convert_to_pdf(png_paths)
        print(f"✅ Created PDF: {os.path.basename(pdf_path)}")
    except Exception as e:
        print(f"❌ Error creating PDF: {str(e)}")
        pdf_path = None
    
    # Create ZIP archive
    try:
        all_files = png_paths
        if pdf_path:
            all_files.append(pdf_path)
        
        zip_path = cert_generator.create_zip_archive(all_files)
        print(f"✅ Created ZIP archive: {os.path.basename(zip_path)}")
        
        return zip_path
    except Exception as e:
        print(f"❌ Error creating ZIP archive: {str(e)}")
        return None

# Create PDF and archive
archive_path = create_pdf_and_archive()

if archive_path:
    print(f"\n📦 Archive created successfully: {archive_path}")
    print("💾 You can download it from the file browser or Google Drive")

## 📥 Download Generated Files

Download individual certificates or the complete archive.

In [ ]:
# Download generated files
def download_certificates():
    print("📥 Available files for download:")
    
    # List all files in output directory
    output_files = os.listdir(output_path)
    
    if not output_files:
        print("❌ No files available for download")
        return
    
    # Categorize files
    png_files = [f for f in output_files if f.endswith('.png')]
    pdf_files = [f for f in output_files if f.endswith('.pdf')]
    zip_files = [f for f in output_files if f.endswith('.zip')]
    
    print(f"📊 PNG Certificates: {len(png_files)}")
    print(f"📄 PDF Files: {len(pdf_files)}")
    print(f"📦 ZIP Archives: {len(zip_files)}")
    
    # Show recent files
    print("\n📋 Recent files:")
    for i, file in enumerate(output_files[-10:]):
        file_path = f'{output_path}/{file}'
        file_size = os.path.getsize(file_path) / 1024  # KB
        print(f"{i+1}. {file} ({file_size:.1f} KB)")
    
    # Download the most recent ZIP file if available
    if zip_files:
        latest_zip = max(zip_files, key=lambda x: os.path.getctime(f'{output_path}/{x}'))
        zip_path = f'{output_path}/{latest_zip}'
        
        print(f"\n📦 Downloading latest archive: {latest_zip}")
        try:
            files.download(zip_path)
            print("✅ Download started!")
        except Exception as e:
            print(f"❌ Download error: {str(e)}")
            print(f"💡 You can manually download from: {zip_path}")
    else:
        print("\n💡 No ZIP archive found. Create an archive first using the previous cells.")

# Download certificates
download_certificates()

## 🎨 Template Customization

Customize certificate templates with your own design elements.

In [ ]:
# Template customization functions
def create_custom_template(name, title, subtitle, colors, layout='standard'):
    """
    Create a custom certificate template
    
    Args:
        name: Template identifier
        title: Main title (e.g., "CERTIFICATE")
        subtitle: Subtitle (e.g., "OF COMPLETION")
        colors: Dict with 'primary', 'secondary', 'accent' colors
        layout: Layout style ('standard', 'modern', 'classic')
    """
    width, height = 1200, 800
    img = Image.new('RGB', (width, height), color='white')
    draw = ImageDraw.Draw(img)
    
    # Layout-specific designs
    if layout == 'modern':
        # Modern design with geometric elements
        draw.rectangle([(0, 0), (width, 100)], fill=colors['primary'])
        draw.rectangle([(0, height-100), (width, height)], fill=colors['primary'])
        
        # Side decorations
        for i in range(0, height, 50):
            draw.rectangle([(0, i), (20, i+25)], fill=colors['accent'])
            draw.rectangle([(width-20, i), (width, i+25)], fill=colors['accent'])
    
    elif layout == 'classic':
        # Classic design with ornate borders
        border_width = 30
        # Outer border
        draw.rectangle([(border_width, border_width), 
                       (width-border_width, height-border_width)], 
                      outline=colors['primary'], width=8)
        # Inner border
        draw.rectangle([(border_width+15, border_width+15), 
                       (width-border_width-15, height-border_width-15)], 
                      outline=colors['secondary'], width=3)
        
        # Corner decorations
        corner_size = 50
        corners = [(border_width, border_width), 
                   (width-border_width-corner_size, border_width),
                   (border_width, height-border_width-corner_size),
                   (width-border_width-corner_size, height-border_width-corner_size)]
        
        for corner in corners:
            draw.rectangle([corner, (corner[0]+corner_size, corner[1]+corner_size)], 
                          fill=colors['accent'])
    
    else:  # standard layout
        # Standard design
        border_width = 20
        draw.rectangle([(border_width, border_width), 
                       (width-border_width, height-border_width)], 
                      outline=colors['primary'], width=5)
        draw.rectangle([(border_width+10, border_width+10), 
                       (width-border_width-10, height-border_width-10)], 
                      outline=colors['accent'], width=2)
    
    # Add title and subtitle placeholders
    draw.text((width//2, 120), title, font=None, fill=colors['primary'], anchor='mm')
    draw.text((width//2, 170), subtitle, font=None, fill=colors['secondary'], anchor='mm')
    
    # Add standard placeholders
    draw.text((width//2, 250), "This is to certify that", font=None, fill='black', anchor='mm')
    draw.text((width//2, 350), "[PARTICIPANT_NAME]", font=None, fill=colors['accent'], anchor='mm')
    draw.text((width//2, 450), "[MAIN_CONTENT]", font=None, fill=colors['primary'], anchor='mm')
    draw.text((width//2, 550), "[DATE]", font=None, fill='black', anchor='mm')
    
    # Save template
    template_path = f'{templates_path}/{name}_template.png'
    img.save(template_path)
    
    return template_path

# Create custom templates
def create_custom_templates():
    print("🎨 Creating Custom Templates...")
    
    custom_templates = {
        'modern_blue': {
            'title': 'CERTIFICATE',
            'subtitle': 'OF EXCELLENCE',
            'colors': {'primary': '#1E3A8A', 'secondary': '#3B82F6', 'accent': '#60A5FA'},
            'layout': 'modern'
        },
        'classic_gold': {
            'title': 'CERTIFICATE',
            'subtitle': 'OF RECOGNITION',
            'colors': {'primary': '#92400E', 'secondary': '#D97706', 'accent': '#F59E0B'},
            'layout': 'classic'
        },
        'elegant_purple': {
            'title': 'CERTIFICATE',
            'subtitle': 'OF ACHIEVEMENT',
            'colors': {'primary': '#6B21A8', 'secondary': '#9333EA', 'accent': '#A855F7'},
            'layout': 'standard'
        }
    }
    
    created_templates = []
    
    for name, config in custom_templates.items():
        template_path = create_custom_template(
            name, 
            config['title'], 
            config['subtitle'], 
            config['colors'], 
            config['layout']
        )
        created_templates.append(template_path)
        print(f"✅ Created custom template: {name}")
    
    return created_templates

# Create custom templates
custom_templates = create_custom_templates()
print(f"\n🎨 Created {len(custom_templates)} custom templates")
print("💡 You can now use these template names: 'modern_blue', 'classic_gold', 'elegant_purple'")

## 📊 Summary and Usage Guide

Complete guide on how to use this certificate generator.

In [ ]:
# Generate usage summary
def generate_usage_summary():
    print("📊 CERTIFICATE GENERATOR - USAGE SUMMARY")
    print("=" * 50)
    
    # Check current status
    templates = [f for f in os.listdir(templates_path) if f.endswith('.png')]
    fonts = [f for f in os.listdir(fonts_path) if f.endswith('.ttf')]
    data_files = [f for f in os.listdir(data_path) if f.endswith(('.xlsx', '.csv'))]
    output_files = [f for f in os.listdir(output_path)]
    
    print(f"📁 Available Templates: {len(templates)}")
    print(f"🔤 Available Fonts: {len(fonts)}")
    print(f"📊 Sample Data Files: {len(data_files)}")
    print(f"📄 Generated Files: {len(output_files)}")
    
    print("\n🎯 AVAILABLE CERTIFICATE TYPES:")
    print("1. completion - Certificate of Completion")
    print("2. achievement - Certificate of Achievement")
    print("3. participation - Certificate of Participation")
    print("4. modern_blue - Custom Modern Blue Design")
    print("5. classic_gold - Custom Classic Gold Design")
    print("6. elegant_purple - Custom Elegant Purple Design")
    
    print("\n📋 REQUIRED DATA COLUMNS:")
    print("Completion: Participant_Name, Course_Name, Date")
    print("Achievement: Participant_Name, Course_Name, Grade, Date")
    print("Participation: Participant_Name, Event_Name, Date")
    
    print("\n🚀 QUICK START GUIDE:")
    print("1. Use sample data: Run cells 9-10 to generate from provided Excel files")
    print("2. Manual entry: Modify cell 8 with your data and run")
    print("3. Upload file: Use cell 11 to upload your Excel/CSV file")
    print("4. Process uploaded: Use cell 12 with correct template type")
    print("5. Create PDF/ZIP: Run cell 13 to package all certificates")
    print("6. Download: Run cell 14 to download the archive")
    
    print("\n💡 CUSTOMIZATION OPTIONS:")
    print("- Modify colors, fonts, and positions in CertificateGenerator class")
    print("- Create new templates using create_custom_template function")
    print("- Add new certificate types by extending the configs dictionary")
    
    print("\n📁 FILE LOCATIONS:")
    print(f"Templates: {templates_path}")
    print(f"Fonts: {fonts_path}")
    print(f"Data: {data_path}")
    print(f"Output: {output_path}")
    
    print("\n✅ SYSTEM STATUS: All components initialized successfully!")
    print("🔄 Ready to generate certificates!")

# Show usage summary
generate_usage_summary()

## 🎉 Example: Complete Workflow

This cell demonstrates a complete workflow from data preparation to final download.

In [ ]:
# Complete workflow example
def complete_workflow_example():
    print("🎉 COMPLETE WORKFLOW DEMONSTRATION")
    print("=" * 40)
    
    # Step 1: Create example data
    print("📊 Step 1: Creating example data...")
    example_data = pd.DataFrame({
        'Participant_Name': ['Alice Johnson', 'Bob Smith', 'Carol Davis'],
        'Course_Name': ['AI Engineering Masterclass', 'Python for Data Science', 'Machine Learning Bootcamp'],
        'Date': ['2024-10-03', '2024-10-03', '2024-10-03'],
        'Instructor': ['Dr. AI Expert', 'Prof. Python', 'ML Guru']
    })
    
    # Save example data
    example_file = f'{data_path}/workflow_example.xlsx'
    example_data.to_excel(example_file, index=False)
    print(f"✅ Created example file: {os.path.basename(example_file)}")
    print(example_data)
    
    # Step 2: Generate certificates
    print("\n🎯 Step 2: Generating certificates...")
    generated_files = cert_generator.generate_from_excel(example_file, 'completion')
    print(f"✅ Generated {len(generated_files)} certificates")
    
    # Step 3: Create PDF
    print("\n📄 Step 3: Creating PDF compilation...")
    pdf_path = cert_generator.convert_to_pdf(generated_files)
    print(f"✅ Created PDF: {os.path.basename(pdf_path)}")
    
    # Step 4: Create ZIP archive
    print("\n📦 Step 4: Creating downloadable archive...")
    all_files = generated_files + [pdf_path]
    zip_path = cert_generator.create_zip_archive(all_files, 'workflow_example.zip')
    print(f"✅ Created archive: {os.path.basename(zip_path)}")
    
    # Step 5: Summary
    print("\n📋 Step 5: Workflow Summary")
    print(f"📊 Input: {len(example_data)} participants")
    print(f"🎯 Output: {len(generated_files)} PNG certificates")
    print(f"📄 PDF: 1 compiled document")
    print(f"📦 Archive: 1 downloadable ZIP file")
    
    archive_size = os.path.getsize(zip_path) / 1024  # KB
    print(f"💾 Archive size: {archive_size:.1f} KB")
    
    print("\n🎉 Workflow completed successfully!")
    print(f"📥 Download your certificates: {zip_path}")
    
    return zip_path

# Run complete workflow
workflow_archive = complete_workflow_example()

# Offer download
try:
    print("\n🔽 Starting download...")
    files.download(workflow_archive)
    print("✅ Download initiated!")
except:
    print(f"💡 Manual download available at: {workflow_archive}")